# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder).

In [1]:
model_patch_size = 8
verbose          = 0
reset_kernel     = False

In [2]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(



Octave is ready <oct2py.core.Oct2Py object at 0x7f33c6b6a350>

















In [3]:
import torch
torch.cuda.set_device(0)

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [4]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [5]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/zeroshot-moment-small-embedding:latest
input_ar: None
cpu: False


### Show configuration artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [6]:
entity = 'mi-santamaria'
project = 'deepvats'
folder = entity+'/'+project+'/'
model_family = 'zeroshot-moment'
task = 'embedding'
dataset = 'gtrends_kohls'
dataset_version = 'v2'
enc_artifact_dataset = folder + dataset + ':' + dataset_version
enc_artifact_small = folder + model_family + '-small-' + task + ':v0'
enc_artifact_base  =  folder + model_family + '-base-' + task + ':v0'
enc_artifact_large = folder + model_family + '-large-' + task + ':v0'

In [7]:
print("Getting dataset artifact: ", enc_artifact_dataset)
df_artifact = wandb_api.artifact(enc_artifact_dataset, type = 'dataset')
print("Getting small artifact: ", enc_artifact_small)
enc_artifact_small = wandb_api.artifact(enc_artifact_small, type='learner')
print("Getting base artifact: ", enc_artifact_base)
enc_artifact_base  = wandb_api.artifact(enc_artifact_base, type='learner')
print("Getting large artifact: ", enc_artifact_large)
enc_artifact_large = wandb_api.artifact(enc_artifact_large, type='learner')

Getting dataset artifact:  mi-santamaria/deepvats/gtrends_kohls:v2
Getting small artifact:  mi-santamaria/deepvats/zeroshot-moment-small-embedding:v0
Getting base artifact:  mi-santamaria/deepvats/zeroshot-moment-base-embedding:v0
Getting large artifact:  mi-santamaria/deepvats/zeroshot-moment-large-embedding:v0


In [8]:
print(df_artifact.name)
df = df_artifact.to_df()
display(df.head)
print(df.shape)

gtrends_kohls:v2


wandb:   1 of 1 files downloaded.  


<bound method NDFrame.head of               volume
2004-01-01  0.010417
2004-01-08  0.010417
2004-01-15  0.010417
2004-01-22  0.000000
2004-01-29  0.000000
...              ...
2012-05-03  0.322917
2012-05-10  0.312500
2012-05-17  0.281250
2012-05-24  0.291667
2012-05-31  0.322917

[440 rows x 1 columns]>

(440, 1)


In [9]:
print(enc_artifact_small.name)
enc_learner_small = enc_artifact_small.to_obj()

zeroshot-moment-small-embedding:v0


wandb: Downloading large artifact zeroshot-moment-small-embedding:v0, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


In [10]:
print(enc_artifact_base.name)
enc_learner_base  = enc_artifact_base.to_obj()

zeroshot-moment-base-embedding:v0


wandb: Downloading large artifact zeroshot-moment-base-embedding:v0, 432.97MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.3


In [11]:
print(enc_artifact_large.name)
enc_learner_large = enc_artifact_large.to_obj()

zeroshot-moment-large-embedding:v0


wandb: Downloading large artifact zeroshot-moment-large-embedding:v0, 1321.42MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:4.0


In [12]:
def count_parameters(model):
    #return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())
print(count_parameters(enc_learner_small))
print(count_parameters(enc_learner_base))
print(count_parameters(enc_learner_large))

35341512
109641608
341248520


In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())
count_parameters(enc_learner_large)

341248520

In [14]:
from fastai.losses import MSELossFlat
from dvats.encoder import MAELossFlat, EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE

In [16]:
enc_input, _ = SlidingWindow(window_len=17, stride=2, get_y=[])(df)
enc_input.shape

(212, 1, 17)

In [17]:
#| export
import dvats.config as cfg_

In [18]:
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)

In [19]:
print(config['batch_size'])
print(config['r'])
print(config['analysis_mode'])

16
0.4
online


In [20]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": config['batch_size'],
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": 0.8, # Comprobando si el None es el problema
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": config['r'], #use enc_run lr,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": None,
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 5, #2.5*enc_input.shape[0]/100,
    "print_to_path": False,
    "print_path": "~/data/logs.txt",
    "print_mode": "w",
    "use_moment_masks": False,
    "mask_stateful": config['mask_stateful'],
    "mask_future": config['mask_future'],
    "mask_sync": config['mask_sync'],
    "analysis_mode": config['analysis_mode'],
    "use_wandb": config['use_wandb'],
    "norm_by_sample": config['norm_by_sample'],
    "norm_use_single_batch": config['norm_use_single_batch'],
    "show_plot": True,
    "metrics": [EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE],
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None
}

In [29]:
import pandas as pd
results = pd.DataFrame(
    columns = [
        'model size',
        'n_epochs',
        'dataset_percent',
        'maskared_percent',
        'losses',
        'eval_results_pre',
        'eval_results_post',
        'time'
    ]
)

errors = pd.DataFrame(
    columns = [
        'model size',
        'n_epochs',
        'dataset_percent',
        'maskared_percent',
        'error'
    ]
)

In [35]:
from copy import deepcopy
i = 0
for n_epochs in [5]:
    for dataset_percent in [0.3]:
        for maskared_percent in [0.4]:
            for sizes in [3]:
                print(f"--> epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                print(f" sizes {sizes}")
                result_dict = {'model_size': "small",
                        'n_epochs': n_epochs,
                        'dataset_percent': dataset_percent,
                        'maskared_percent': maskared_percent,
                       }
                error_dict = deepcopy(result_dict)
                result_dict.update({
                        'losses': np.nan,
                        'eval_results_pre': np.nan, 'eval_results_post': np.nan,
                        'time': np.nan
                    })
                error = False
                error_dict.update({'error': ""})
                try:
                    result = fine_tune(
                        enc_learn           = enc_learner_small,
                        window_mask_percent = maskared_percent,
                        training_percent    = dataset_percent,
                        validation_percent  = 0.3, #1-dataset_percent if 1-dataset_percent != 0 else 0.3,
                        num_epochs          = n_epochs,
                        n_window_sizes      = sizes,
                        verbose             = 4,
                        **common_args    
                    )
                    result_dict['losses'] = result[0]
                    result_dict['eval_results_pre'] = result[1]
                    result_dict['eval_results_post'] = result[2]
                    result_dict['time'] = result[4]
                except Exception as e:
                    error = True
                    print("Failure, see errors dataset")
                    error_dict['error'] = e
                    errors = pd.concat([errors, pd.DataFrame([error_dict])], ignore_index=True)                
                results = pd.concat([results, pd.DataFrame([result_dict])], ignore_index=True)                
                print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                if not error: print(f" sizes {sizes} | time: {result[4]} -->")
                    


--> epoch 5, dataset_percent 0.3, mask 0.4
 sizes 3
[4] [ --> _get_encoder ]
[4]  [ _get_encoder ] About to exec _get_enc_input
[4] [ --> _get_enc_input ]
[4]  [ _get_enc_input ] is none enc_input? True
[4]  [ _get_enc_input ] About to get the windows
[4] [ --> windowed_dataset ]
[4]  [ _get_enc_input ] X is a DataFrame, X~(440, 1) | window_sizes 0, n_window_sizes 3
[4]  [ _get_enc_input ] X is a DataFrame | Selecting Fourier's dominant frequences
[4] [ --> Find_dominant_window_sizes_list ]
[4]  [ Find_dominant_window_sizes_list ] X ~ (440, 1)
[4]  [ Find_dominant_window_sizes_list ] Grouping sizes
[4] [Find_dominant_window_sizes_list --> ]
[4]  [ windowed_dataset ] X is a DataFrame | Window sizes: 3
[4]  [ windowed_dataset ] Building the windows
[4]  [ windowed_dataset ] w = 17
[4]  [ windowed_dataset ] w 17 | enc_input~(424, 1, 17) | dss~1
[4]  [ windowed_dataset ] w = 12
[4]  [ windowed_dataset ] w 12 | enc_input~(429, 1, 12) | dss~2
[4]  [ windowed_dataset ] w = 4
[4]  [ windowed_d





  0% 0/7 [00:00<?, ?it/s]



 14% 1/7 [00:01<00:06,  1.08s/it]



 57% 4/7 [00:01<00:00,  4.15it/s]



100% 7/7 [00:01<00:00,  5.20it/s]

[4]  [ fine_tune_moment_single ] End timer -->
[4]  [ fine_tune_moment_single ] Start: 1737480032.1250749 | End: 1737480033.5149786 | Duration: 1.39 seconds
[4]  [ fine_tune_moment_single ] Start timer
[4]  [ fine_tune_moment_single ] Train | wlen 17
[3] fine_tune_moment_train_ | Training loop
[3] fine_tune_moment_train | Fine tune loop | print_to_path False | batch_masks~tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 






  0% 0/30 [00:00<?, ?it/s]

[3] fine_tune_moment_train | num_epochs 5 | n_batches 6
[3] fine_tune_moment_train | batch 0 ~ torch.Size([16, 1, 17]) | epoch 0 | train 0 of 30 | Before loop step
[3] fine_tune_moment_train_loop_step_ | Fine tune loop | batch ~ torch.Size([16, 1, 17]) | batch_masks ~ torch.Size([16, 17])
[3] fine_tune_moment_train_loop_step_ | Fine tune loop | window_mask_percent 0.4 | batch ~ torch.Size([16, 1, 17])
[3] fine_tune_moment_train_loop_step_ | Fine tune loop | o ~ torch.Size([16, 17]) | stateful = False | sync = False | r = 0.4
[3] fine_tune_moment_train_loop_step_ | Fine tune loop | Before shape adjustment | batch ~ torch.Size([16, 1, 17]) | batch_masks ~ torch.Size([16, 17]) | mask ~ torch.Size([16, 17])
[3] fine_tune_moment_train_loop_step_ | Fine tune loop | batch ~ torch.Size([16, 1, 17]) | batch_masks ~ torch.Size([16, 17]) | mask ~ torch.Size([16, 17])
[3] fine_tune_moment_train_loop_step_ | sure_eval_moment | bcuda:0 | mcuda:0 | bmcuda:0
[3] ---> sure_eval_moment
[3] sure_eval_mom

...

[3] sure_eval_moment | Trial 5 | device 0 | input_mask~torch.Size([4, 4]) device: cuda:0
[3] sure_eval_moment | Trial 5 | device 0 | mask device~torch.Size([4, 4]): cuda:0
[3] sure_eval_moment | Trial 5 | device 0 | y~torch.Size([4, 1, 4]) device: cuda:0
[3] sure_eval_moment | Trial 5 | About to pad X (encoder input) | exception maximum size for tensor at dimension 2 is 4 but size is 8 | padd step: 100
[3] sure_eval_moment | Trial 5 | y ~ torch.Size([4, 1, 4])
[3] Not the usual error. No padding, just fail
[3] sure_eval_moment | output <class 'NoneType'> -->
[3] fine_tune_moment_train_loop_step_ | Execution failed | Output none 
[3] fine_tune_moment_train | batch 5 ~ torch.Size([4, 1, 4]) | epoch 4 | train 9 of 30 | Loss backward | After loop step 


Traceback (most recent call last):
  File "/home/macu/work/dvats/encoder.py", line 794, in sure_eval_moment
    output = enc_learn(x_enc = y, input_mask = input_mask, mask = mask)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 227, in __call__
    return self.forward(*args, **kwargs)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 566, in forward
    return self.reconstruction(
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/momentfm/models/moment.py", line 299, in reconstruction
    x_enc = self.tokenizer(x=x_enc)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*a

[3] fine_tune_moment_train | -->
[4]  [ fine_tune_moment_single ] End timer -->
[4]  [ fine_tune_moment_single ] Start: 1737480039.26312 | End: 1737480040.9276917 | Duration: 1.66 seconds
[4]  [ fine_tune_moment_single ] fine_tune_moment_single | Eval Post | wlen 17
[4]  [ fine_tune_moment_single ] Start timer







  0% 0/7 [00:00<?, ?it/s]

Failure, see errors dataset
epoch 5, dataset_percent 0.3, mask 0.4


In [33]:
results.head()

,model size,n_epochs,dataset_percent,maskared_percent,losses,eval_results_pre,eval_results_post,time,model_size
0,NaN,5,0.3,0.4,NaN,NaN,NaN,NaN,small
1,NaN,5,0.3,0.4,NaN,NaN,NaN,NaN,small


In [34]:
errors.head()

,model size,n_epochs,dataset_percent,maskared_percent,error,model_size
0,NaN,5,0.3,0.4,maximum size for tensor at dimension 2 is 4 but size is 8,small
1,NaN,5,0.3,0.4,maximum size for tensor at dimension 2 is 4 but size is 8,small


In [23]:
#| export
if verbose > 0: print("Execution ended")
from dvats.imports import beep
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)

In [24]:
#| hide
if reset_kernel:
    import os
    os._exit(00)